# fbiPy - Prisoners in the US
----

## Dataset: FBI

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
import json
# Import API key
from config import apikey

### Table 1: Incidents reported to the NCVS

### Perform API Calls
* Get the personal victimization counts of incidents reported to the NCVS in 2019.


In [3]:
# Config information.
base_url = "https://api.usa.gov/crime/fbi/sapi/"
format_type = "json"

In [ ]:
endpoint = "api/agencies"

# Build query URL
query_url = f"{base_url}{endpoint}?api_key={apikey}"
# print(query_url)
# Call API and get response
response = requests.get(query_url)
data_json = response.json()
agencies_df = pd.DataFrame()
for key_level1 in data_json:
#     print(f'Level 1 {key_level1}')
    for key_level2 in data_json[key_level1]:
#         print(f'Level 2 {key_level2}')
#         print(f"Agency: {data_json[key_level1][key_level2]['agency_name']}")
#         print(f"State: {data_json[key_level1][key_level2]['state_abbr']}")
        agency_dict = data_json[key_level1][key_level2]
#         print(agency_dict)
        agency_df = pd.DataFrame(agency_dict, index=[0])
        print (agency_df)
        agencies_df.append(agency_df) 
print(agencies_df)

         ori                                       agency_name  \
0  HI0010000  Hawaii Police Department County Sheriff's Office   

  agency_type_name state_name state_abbr division_name region_name  \
0           County     Hawaii         HI       Pacific        West   

  region_desc county_name  nibrs   latitude  longitude nibrs_start_date  
0   Region IV      HAWAII  False  19.716698 -155.08688             None  
         ori                                     agency_name agency_type_name  \
0  HI0050000  Maui Police Department County Sheriff's Office           County   

  state_name state_abbr division_name region_name region_desc county_name  \
0     Hawaii         HI       Pacific        West   Region IV        MAUI   

   nibrs   latitude  longitude nibrs_start_date  
0  False  20.887321 -156.48763             None  
         ori                 agency_name agency_type_name state_name  \
0  HI0020000  Honolulu Police Department             City     Hawaii   

  state_abbr di

         ori    agency_name agency_type_name state_name state_abbr  \
0  DE002159E  Amtrak Police            Other   Delaware         DE   

    division_name region_name region_desc county_name  nibrs latitude  \
0  South Atlantic       South  Region III  NEW CASTLE   True     None   

  longitude nibrs_start_date  
0      None       01/01/2001  
         ori                   agency_name agency_type_name state_name  \
0  DE0010200  Harrington Police Department             City   Delaware   

  state_abbr   division_name region_name region_desc county_name  nibrs  \
0         DE  South Atlantic       South  Region III        KENT   True   

    latitude  longitude nibrs_start_date  
0  38.924095  -75.57957       01/01/2001  
         ori                       agency_name agency_type_name state_name  \
0  DE0031200  Fenwick Island Police Department             City   Delaware   

  state_abbr   division_name region_name region_desc county_name  nibrs  \
0         DE  South Atlantic    

0  39.575915 -75.644132       01/01/2001  
         ori                   agency_name agency_type_name state_name  \
0  DE0020600  Wilmington Police Department             City   Delaware   

  state_abbr   division_name region_name region_desc county_name  nibrs  \
0         DE  South Atlantic       South  Region III  NEW CASTLE   True   

   latitude  longitude nibrs_start_date  
0  39.73873 -75.548965       01/01/2001  
         ori      agency_name    agency_type_name state_name state_abbr  \
0  DE002500X  Animal Welfare:  Other State Agency   Delaware         DE   

    division_name region_name region_desc county_name  nibrs latitude  \
0  South Atlantic       South  Region III  NEW CASTLE   True     None   

  longitude nibrs_start_date  
0      None       09/01/2015  
         ori                   agency_name agency_type_name state_name  \
0  DE0031700  Ocean View Police Department             City   Delaware   

  state_abbr   division_name region_name region_desc county_name

0 -104.261619             None  
         ori                     agency_name agency_type_name state_name  \
0  TX1580000  Marion County Sheriff's Office           County      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III      MARION   True   

    latitude  longitude nibrs_start_date  
0  32.798182 -94.356853       05/01/2019  
         ori                     agency_name agency_type_name state_name  \
0  TX2470000  Wilson County Sheriff's Office           County      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III      WILSON   True   

    latitude  longitude nibrs_start_date  
0  29.173883 -98.086734       05/01/2019  
         ori                   agency_name agency_type_name state_name  \
0  TX0612500  Double Oak Police Department             City      Texas   

  state_abbr       div

0  -98.18099             None  
         ori               agency_name agency_type_name state_name state_abbr  \
0  TX0670400  Ranger Police Department             City      Texas         TX   

        division_name region_name region_desc county_name  nibrs   latitude  \
0  West South Central       South  Region III    EASTLAND  False  32.471584   

   longitude nibrs_start_date  
0    -98.683             None  
         ori               agency_name agency_type_name state_name state_abbr  \
0  TX0611400  Aubrey Police Department             City      Texas         TX   

        division_name region_name region_desc county_name  nibrs   latitude  \
0  West South Central       South  Region III      DENTON  False  33.304184   

   longitude nibrs_start_date  
0  -96.98627       01/01/2020  
         ori                 agency_name agency_type_name state_name  \
0  TX1260100  Alvarado Police Department             City      Texas   

  state_abbr       division_name region_name region

...

0  30.574388 -95.163137             None  
         ori agency_name agency_type_name state_name state_abbr  \
0  TX2101600      Tenaha             City      Texas         TX   

        division_name region_name region_desc county_name  nibrs latitude  \
0  West South Central       South  Region III      SHELBY   True     None   

  longitude nibrs_start_date  
0      None       01/01/2019  
         ori                   agency_name agency_type_name state_name  \
0  TX1701200  Shenandoah Police Department             City      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III  MONTGOMERY   True   

    latitude  longitude nibrs_start_date  
0  30.299402 -95.504008       04/01/2017  
         ori               agency_name agency_type_name state_name state_abbr  \
0  TX1770200  Roscoe Police Department             City      Texas         TX   

        division_name region_name region_desc co

         ori                         agency_name agency_type_name state_name  \
0  TX1471100  Independent School District: Mexia            Other      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III   LIMESTONE  False   

   latitude  longitude nibrs_start_date  
0   31.6865 -96.476814             None  
         ori                      agency_name agency_type_name state_name  \
0  TX1460000  Liberty County Sheriff's Office           County      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III     LIBERTY   True   

    latitude  longitude nibrs_start_date  
0  30.162189 -94.822682       01/01/2018  
         ori            agency_name agency_type_name state_name state_abbr  \
0  TX2120100  Arp Police Department             City      Texas         TX   

        division_name region_name regio

0  29.468704 -97.477738             None  
         ori                     agency_name agency_type_name state_name  \
0  TX2210000  Taylor County Sheriff's Office           County      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III      TAYLOR  False   

    latitude  longitude nibrs_start_date  
0  32.295054 -99.893644             None  
         ori               agency_name agency_type_name state_name state_abbr  \
0  TX0031300  Hudson Police Department             City      Texas         TX   

        division_name region_name region_desc county_name  nibrs  latitude  \
0  West South Central       South  Region III    ANGELINA  False  31.32794   

   longitude nibrs_start_date  
0  -94.78142             None  
         ori                 agency_name agency_type_name state_name  \
0  TX1070200  Malakoff Police Department             City      Texas   

  state_abbr       division_na

         ori               agency_name       agency_type_name state_name  \
0  TX2400500  Laredo Community College  University or College      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III        WEBB  False   

   latitude  longitude nibrs_start_date  
0  27.50878  -99.51866             None  
         ori                   agency_name agency_type_name state_name  \
0  TX1210200  Kirbyville Police Department             City      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III      JASPER  False   

    latitude  longitude nibrs_start_date  
0  30.660553  -93.89276             None  
         ori               agency_name agency_type_name state_name state_abbr  \
0  TX1701400  Willis Police Department             City      Texas         TX   

        division_name region_name region_desc c

0      None       01/01/2018  
         ori               agency_name agency_type_name state_name state_abbr  \
0  TX0572500  Wilmer Police Department             City      Texas         TX   

        division_name region_name region_desc county_name  nibrs  latitude  \
0  West South Central       South  Region III      DALLAS  False   32.5892   

   longitude nibrs_start_date  
0  -96.68492             None  
         ori                      agency_name agency_type_name state_name  \
0  TX1680100  Colorado City Police Department             City      Texas   

  state_abbr       division_name region_name region_desc county_name  nibrs  \
0         TX  West South Central       South  Region III    MITCHELL  False   

    latitude  longitude nibrs_start_date  
0  32.303781 -100.92458             None  
         ori                        agency_name agency_type_name state_name  \
0  TX1016700  Independent School District: Katy            Other      Texas   

  state_abbr       divisio

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# prisoners_df.describe()

## Plotting the Data


### x_values vs. y_values Plot

In [ ]:
# # Build a scatter plot
# plt.scatter(x_values, y_values, marker="o")

# # Incorporate the other graph properties
# plt.title(f"...")
# plt.ylabel("...")
# plt.xlabel("...")
# plt.grid(True)
# clb = plt.colorbar()

# # Save the figure
# plt.savefig("../output_data/....png")

# # Show plot
# plt.show()

## Linear Regression

In [ ]:
# # Perform a linear regression on temperature vs. latitude
# x_values = ...
# y_values = ...

# # Calculate the line equation using linear regression function
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
# regress_values = x_values * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# # Plot original data using Scatter type
# plt.scatter(x_values,y_values, label='original data')
# clb = plt.colorbar()

# # Plot fitted line using Line type
# plt.plot(x_values,regress_values,"r-", label='fitted line')

# # Add line equation to plot
# plt.annotate(line_eq,(4,-40),fontsize=15,color="red")

# # Set title, xlabel, ylabel and legend
# plt.title('...\n')
# plt.xlabel('...')
# plt.ylabel('...')
# plt.legend()

# # Show r-squared value
# r_latex = r'$r^2 = $'
# r_value = f'{r_latex}{round(rvalue**2,4)}'
# plt.annotate(r_value,(4,-10),fontsize=15,color="green")

# # Save the figure
# plt.savefig("../output_data/...png")

# # Show plot
# plt.show()